# LDED Audiovisual Fusion 

Author: Chen Lequn.
Created on 13 Sep 2023.

- Material: Maraging Steel 300
- Process: Robotic Laser-directed energy deposition
- Recorded data: position, veolocity, coaxial ccd features, audio feature
- Quality labels generated: keyhole pores, cracks, defect-free

### Notebook 4: ML modelling - baseline 1 (feature-based audio visual fusion model)
- Using the handcrafted features from video and audio stream
- Vision features: melt pool geometric features, including width, length, moment of area, convex hull, etc.
- Audio features: spectral centroid, spectral bandwidth, flux, etc.
- __Modelling__: Once the features are ready, they can be used to train a machine learning model.
- __Evaluate__ the predictions of the validation data by calculating the f1 score and showing a confusion matrix

In [2]:
# from sklearnex import patch_sklearn, config_context
# import dpctl
# patch_sklearn()

In [3]:
import pandas as pd
import numpy as np
import scipy as sp
import os
import math
import sys
import itertools
from pandas import DataFrame

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import gca
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
%matplotlib inline
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier, RUSBoostClassifier
from collections import defaultdict
import pandas as pd
from sklearn.preprocessing import StandardScaler
# For K Nearest Neighbours:
from sklearn import neighbors
# For support vector machine:
from sklearn import svm
# For Gaussian process:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.gaussian_process.kernels import RBF
# For neural network:
from sklearn.neural_network import MLPClassifier
# For decision tree:
from sklearn import tree
# For plotting the decision tree structure:
# import graphviz
import pickle
# For ensemble methods: random forest ad AdaBoost
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# For Naive Bayes method:
from sklearn.naive_bayes import GaussianNB
# For logistic regression:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

from imblearn.ensemble import BalancedRandomForestClassifier, RUSBoostClassifier


from pprint import pprint
from scipy.interpolate import griddata
from pylab import * # For adjusting frame width only
from sklearn.preprocessing import OneHotEncoder
import sklearn.gaussian_process as gp 
import sklearn.model_selection as ms
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_validate
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.utils import shuffle
from sklearn.linear_model import Ridge 
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import scale
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn import svm
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.utils import resample, class_weight
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
import xgboost as xgb

from sklearn.datasets import load_digits
# Use grid search with cross validation to select ML model hyper-parameters:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV  
from sklearn import metrics
from sklearn.metrics import classification_report

In [4]:
# For plotting
import plotly.io as pio
import plotly.graph_objects as go

import numpy as np
import matplotlib.pyplot as plt

# For ML model saving
import pickle
# sns.set_theme(style="whitegrid")
sns.set_theme(style="white", palette=None)

In [5]:
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 2.50

In [6]:
import sys
from utils import *
import glob
import os
# import utils
FRAME_SIZE = 2048
HOP_LENGTH = 512

In [7]:
### Function to plot confusion matrix
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure(figsize=(8, 8))
    im_ratio = cm.shape[1]/cm.shape[0]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=18, pad=12)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90, fontsize=12)
    plt.yticks(tick_marks, classes, fontsize=12)

    fmt = '.3f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 fontsize = 16, 
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('Ground Truth', fontsize=20, labelpad =12)
    plt.xlabel('Predicted', fontsize=20, labelpad =12)
    plt.xticks(fontsize=16,  rotation=30, ha='right')
    plt.yticks(fontsize=16)
    cbar = plt.colorbar(orientation="vertical", pad=0.1, ticks=[0, 0.5, 1], fraction=0.045*im_ratio)
    cbar.ax.tick_params(labelsize=14)
    cbar.ax.set_title('Accuracy',fontsize=16, pad = 12)
    plt.tight_layout()

    
## Define function to get the confusion matrix and print out the plot as well
def conf_matrix(y_true, y_pred, classes=["Laser-off",'Defect-free','Cracks','Keyhole pores'] ):
    cm = confusion_matrix(y_true, y_pred)
    
    # convert to percentage and plot the confusion matrix
    cm_pct = cm.astype(float) / cm.sum(axis =1)[:,np.newaxis]
    
    # classes = le.classes_
    print(cm)
    plot_confusion_matrix(cm_pct, classes)

In [8]:
def show_confusion_matrix(y_true, y_pred, classes=None, classes_categorical=None, normalize=None, figsize=(10, 10), 
                          dpi=600, fontsize=10, axis_fontsize=14, tick_size=12):
    cm = confusion_matrix(y_true, y_pred, normalize=normalize, labels=classes)
    
    if normalize == 'true':
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    cax = ax.matshow(cm, cmap=plt.cm.Blues)
    
    cbar = plt.colorbar(cax, ax=ax, shrink=0.65)
    
    if classes:
        tick_marks = np.arange(len(classes))
        plt.xticks(tick_marks, classes_categorical, rotation=45, fontsize=tick_size, ha='left')
        plt.yticks(tick_marks, classes_categorical, fontsize=tick_size)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], '.2f' if normalize else 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black",
                 fontsize=fontsize)
    
    ax.set_xlabel('Predicted label', fontsize=axis_fontsize)
    ax.set_ylabel('True label', fontsize=axis_fontsize)

In [9]:
PROJECT_ROOT_DIR = "../"
IMAGE_PATH = os.path.join(PROJECT_ROOT_DIR, "result_images", 'defect classification', 'class_name_v3', 'ML-baseline', 'audio_visual')
model_path = os.path.join(PROJECT_ROOT_DIR, "trained_models", 'defect classification', 'class_name_v3', 'ML-baseline', 'audio_visual')
os.makedirs(IMAGE_PATH, exist_ok=True)
os.makedirs(model_path, exist_ok=True)

Multimodal_dataset_PATH = "/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset"
Dataset_path = os.path.join(Multimodal_dataset_PATH, f'25Hz')
                            

## function for automatically save the diagram/graph into the folder 
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGE_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 2.50

## Import Extracted features

In [10]:
# Read the data back from the HDF5 file into a new DataFrame
df_multimodal = pd.read_hdf(os.path.join(Dataset_path, 'spatiotemporal_fused_multimodal.h5'), key='df')
df_multimodal = df_multimodal.dropna(subset=['class_name_v3'])
df_multimodal

sample index  Time_Stamps      audio_file_name      image_file_name  \
0                 1         0.00      sample_21_1.wav      sample_21_1.jpg   
1                 2         0.04      sample_21_2.wav      sample_21_2.jpg   
2                 3         0.08      sample_21_3.wav      sample_21_3.jpg   
3                 4         0.12      sample_21_4.wav      sample_21_4.jpg   
4                 5         0.16      sample_21_5.wav      sample_21_5.jpg   
...             ...          ...                  ...                  ...   
48461         10982       439.24  sample_26_10982.wav  sample_26_10982.jpg   
48462         10983       439.28  sample_26_10983.wav  sample_26_10983.jpg   
48463         10984       439.32  sample_26_10984.wav  sample_26_10984.jpg   
48464         10985       439.36  sample_26_10985.wav  sample_26_10985.jpg   
48465         10986       439.40  sample_26_10986.wav  sample_26_10986.jpg   

        class_name class_name_v2  Layer number  Sample number class_name_v3  \
0        Laser-off     Laser-off           1.0             21     Laser-off   
1      Defect-free   Defect-free           1.0             21   Defect-free   
2      Defect-free   Defect-free           1.0             21   Defect-free   
3      Defect-free   Defect-free           1.0             21   Defect-free   
4      Defect-free   Defect-free           1.0             21   Defect-free   
...            ...           ...           ...            ...           ...   
48461    Laser-off     Laser-off           NaN             26     Laser-off   
48462    Laser-off     Laser-off           NaN             26     Laser-off   
48463    Laser-off     Laser-off           NaN             26     Laser-off   
48464    Laser-off     Laser-off           NaN             26     Laser-off   
48465    Laser-off     Laser-off           NaN             26     Laser-off   

       rms_energy  ...    center_y  contour_area  contour_length      X  \
0        0.009018  ...    0.000000           0.0        0.000000  0.000   
1        0.019286  ...  239.000000      305442.0     2234.000000  0.000   
2        0.019593  ...  239.000000      305442.0     2234.000000 -0.010   
3        0.030937  ...  230.264496      291865.5     2279.781744 -0.007   
4        0.038329  ...  229.109962      281970.0     2407.847760 -0.111   
...           ...  ...         ...           ...             ...    ...   
48461    0.011000  ...    0.000000           0.0        0.000000  0.000   
48462    0.008347  ...    0.000000           0.0        0.000000  0.000   
48463    0.011081  ...    0.000000           0.0        0.000000  0.000   
48464    0.011044  ...    0.000000           0.0        0.000000  0.000   
48465    0.013110  ...    0.000000           0.0        0.000000  0.000   

           Y      Z     Vx     Vy     Vz  Speed  
0      0.000  0.000  0.000 -0.000  0.000  0.000  
1     -0.000  0.001 -0.074 -0.074  0.000  0.105  
2      0.015 -0.007 -0.051  0.026 -0.051  0.077  
3      0.016  0.010  0.173 -0.222  2.520  2.536  
4      0.123 -0.057 -1.864  1.662 -2.468  3.511  
...      ...    ...    ...    ...    ...    ...  
48461  0.000  0.000 -0.076  0.025  0.000  0.080  
48462  0.000  0.000 -0.076  0.025  0.000  0.080  
48463  0.000  0.000 -0.076  0.025  0.000  0.080  
48464  0.000  0.000 -0.076  0.025  0.000  0.080  
48465  0.000  0.000 -0.076  0.025  0.000  0.080  

[36557 rows x 135 columns]

In [11]:
df_multimodal['class_name_v3'].unique()

['Laser-off', 'Defect-free', 'Laser defocus', 'Keyhole pores']
Categories (4, object): ['Defect-free', 'Keyhole pores', 'Laser defocus', 'Laser-off']

In [12]:
print (df_multimodal.columns)

Index(['sample index', 'Time_Stamps', 'audio_file_name', 'image_file_name',
       'class_name', 'class_name_v2', 'Layer number', 'Sample number',
       'class_name_v3', 'rms_energy',
       ...
       'center_y', 'contour_area', 'contour_length', 'X', 'Y', 'Z', 'Vx', 'Vy',
       'Vz', 'Speed'],
      dtype='object', length=135)


## Constructing labels and input features for training the models

In [13]:
# Label = df_multimodal['class_name'].to_list()
y = df_multimodal['class_name_v3'].to_list()

# Drop non-feature columns and store the result in a variable
# Drop non-feature columns
non_feature_columns = ['sample index', 'Time_Stamps', 'audio_file_name', 'image_file_name',
                       'class_name', 'class_name_v2', 'class_name_v3', 'Layer number', 'Sample number', 'X', 'Y', 'Z', 'Vx', 'Vy', 'Vz', 'Speed']

    
X_all = df_multimodal.drop(columns=non_feature_columns).to_numpy()

redundant_audio_features = ['dynamic_complexity','loudness']


X_audio = df_multimodal[['rms_energy', 'amplitude_envelope_mean',
                           'amplitude_envelope_std', 'zero_crossing_rate', 'dynamic_complexity',
                           'loudness', 'loudness_vickers', 'spectral_centroid_mean',
                           'spectral_centroid_std', 'spectral_complexity_mean',
                           'spectral_complexity_std', 'spectral_contrast_0_mean',
                           'spectral_contrast_0_std', 'spectral_contrast_1_mean',
                           'spectral_contrast_1_std', 'spectral_contrast_2_mean',
                           'spectral_contrast_2_std', 'spectral_contrast_3_mean',
                           'spectral_contrast_3_std', 'spectral_contrast_4_mean',
                           'spectral_contrast_4_std', 'spectral_contrast_5_mean',
                           'spectral_contrast_5_std', 'spectral_valley_0_mean',
                           'spectral_valley_0_std', 'spectral_valley_1_mean',
                           'spectral_valley_1_std', 'spectral_valley_2_mean',
                           'spectral_valley_2_std', 'spectral_valley_3_mean',
                           'spectral_valley_3_std', 'spectral_valley_4_mean',
                           'spectral_valley_4_std', 'spectral_valley_5_mean',
                           'spectral_valley_5_std', 'spectral_decrease_mean',
                           'spectral_decrease_std', 'spectral_energy_mean', 'spectral_energy_std',
                           'spectral_energy_band_ratio_mean', 'spectral_energy_band_ratio_std',
                           'spectral_flatness_mean', 'spectral_flatness_std', 'spectral_flux_mean',
                           'spectral_flux_std', 'spectral_rolloff_mean', 'spectral_rolloff_std',
                           'spectral_strong_peak_mean', 'spectral_strong_peak_std',
                           'spectral_variance_mean', 'spectral_variance_std',
                           'spectral_skewness_mean', 'spectral_skewness_std',
                           'spectral_kurtosis_mean', 'spectral_kurtosis_std',
                           'spectral_crest_factor_mean', 'spectral_crest_factor_std',
                           'mfcc_0_mean', 'mfcc_0_std', 'mfcc_1_mean', 'mfcc_1_std', 'mfcc_2_mean',
                           'mfcc_2_std', 'mfcc_3_mean', 'mfcc_3_std', 'mfcc_4_mean', 'mfcc_4_std',
                           'mfcc_5_mean', 'mfcc_5_std', 'mfcc_6_mean', 'mfcc_6_std', 'mfcc_7_mean',
                           'mfcc_7_std', 'mfcc_8_mean', 'mfcc_8_std', 'mfcc_9_mean', 'mfcc_9_std',
                           'mfcc_10_mean', 'mfcc_10_std', 'mfcc_11_mean', 'mfcc_11_std',
                           'mfcc_12_mean', 'mfcc_12_std']].drop(columns=redundant_audio_features).to_numpy()
                     

X_vision = df_multimodal[['max_contour_area', 'rectangle_angle',
                          'rectangle_width', 'rectangle_height', 'ellipse_angle', 'ellipse_width',
                          'ellipse_height', 'max_hull', 'm00', 'm10', 'm01', 'm20', 'm11', 'm02',
                          'm30', 'm21', 'm12', 'm03', 'mu20', 'mu11', 'mu02', 'mu30', 'mu21',
                          'mu12', 'mu03', 'nu20', 'nu11', 'nu02', 'nu30', 'nu21', 'nu12', 'nu03',
                          'center_x', 'center_y', 'contour_area', 'contour_length']].to_numpy()

# ML Modelling 

In [14]:
all_accuracy_mean_list = []
all_accuracy_std_list = []
vision_accuracy_mean_list = []
vision_accuracy_std_list = []
acoustic_accuracy_mean_list = []
acoustic_accuracy_std_list = []

all_auc_mean = []
all_auc_std = []
vision_auc_mean = []
vision_auc_std = []
acoustic_auc_mean = []
acoustic_auc_std = []

import time

### Train val test split
- Train 80%, Val 10%, Test 10%

In [42]:
y_array = np.array(y)

# First split: Separate out the training set (80% of original)
train_indices, temp_indices = train_test_split(
    np.arange(len(df_multimodal)), test_size=0.2, random_state=0, stratify=y_array)

# Second split: Separate out the validation and test sets (each will be 10% of original)
val_indices, test_indices = train_test_split(
    temp_indices, test_size=0.5, random_state=0, stratify=y_array[temp_indices])

# Check the shape of the indices for train, val, and test splits
len(train_indices), len(val_indices), len(test_indices)

(29245, 3656, 3656)

In [43]:
split_indices = {
    'train_indices': train_indices,
    'val_indices': val_indices,
    'test_indices': test_indices
}

split_indices_path = 'split_indices.pkl'

with open(split_indices_path, 'wb') as f:
    pickle.dump(split_indices, f)

split_indices_path

'split_indices.pkl'

In [44]:
%store train_indices
%store val_indices
%store test_indices

Stored 'train_indices' (ndarray)
Stored 'val_indices' (ndarray)
Stored 'test_indices' (ndarray)


In [38]:
# Create train, val, and test datasets for traditional ML models
X_all_train = X_all[train_indices, :]
Y_train = np.array(y)[train_indices]
X_all_val = X_all[val_indices, :]
Y_val = np.array(y)[val_indices]
X_all_test = X_all[test_indices, :]
Y_test = np.array(y)[test_indices]

# For single modality datasets (audio and vision)
X_audio_train = X_audio[train_indices, :]
X_audio_val = X_audio[val_indices, :]
X_audio_test = X_audio[test_indices, :]

X_vision_train = X_vision[train_indices, :]
X_vision_val = X_vision[val_indices, :]
X_vision_test = X_vision[test_indices, :]

In [39]:
# Label encoding
le = LabelEncoder()
le.fit(y)

Y_train_encoded = le.transform(Y_train)
Y_val_encoded = le.transform(Y_val)
Y_test_encoded = le.transform(Y_test)

In [40]:
print(f"Training set size: {len(Y_train)}")
print(f"Validation set size: {len(Y_val)}")
print(f"Testing set size: {len(Y_test)}")

Training set size: 29245
Validation set size: 7238
Testing set size: 74


In [41]:
le.classes_

array(['Defect-free', 'Keyhole pores', 'Laser defocus', 'Laser-off'],
      dtype='<U13')

In [22]:
Y_train_encoded

array([0, 3, 3, ..., 2, 1, 2])

### Compute class weight

This is for treating the imbalanced class, which is calculating the class weight and use it for weighting the loss function (during training only).

`'balanced'` => n_samples / (n_classes * np.bincount(y))

In [23]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(Y_train),
                                                 y = Y_train)

class_weights_encoded = class_weight.compute_class_weight('balanced',
                                                 classes = np.unique(Y_train_encoded),
                                                 y = Y_train_encoded)


class_names = (le.classes_).tolist()
print(f"class weights: {class_weights}")
print(f"class weights encoded: {class_weights_encoded}")
print(f"class        : {class_names}")

class weights: [1.20231048 1.83101678 1.47226138 0.51469553]
class weights encoded: [1.20231048 1.83101678 1.47226138 0.51469553]
class        : ['Defect-free', 'Keyhole pores', 'Laser defocus', 'Laser-off']


In [24]:
print(f"class weights: {class_weights}")
print(f"class        : {np.unique(le.inverse_transform(Y_train_encoded))}")

class weights: [1.20231048 1.83101678 1.47226138 0.51469553]
class        : ['Defect-free' 'Keyhole pores' 'Laser defocus' 'Laser-off']


### Data standarization

In [25]:
# For "all" modality
scaler_all = preprocessing.StandardScaler()
X_all_train_transformed = scaler_all.fit_transform(X_all_train)
X_all_val_transformed = scaler_all.transform(X_all_val)
X_all_test_transformed = scaler_all.transform(X_all_test)

# For "audio" modality
scaler_audio = preprocessing.StandardScaler()
X_audio_train_transformed = scaler_audio.fit_transform(X_audio_train)
X_audio_val_transformed = scaler_audio.transform(X_audio_val)
X_audio_test_transformed = scaler_audio.transform(X_audio_test)

# For "vision" modality
scaler_vision = preprocessing.StandardScaler()
X_vision_train_transformed = scaler_vision.fit_transform(X_vision_train)
X_vision_val_transformed = scaler_vision.transform(X_vision_val)
X_vision_test_transformed = scaler_vision.transform(X_vision_test)

In [26]:
X_all_train_transformed

array([[ 0.81480926,  1.42071936,  0.69850722, ...,  0.40508304,
         0.12286475,  0.63778654],
       [-0.96847554, -1.03018035, -0.83527196, ..., -0.94187531,
        -0.60054493, -0.89814704],
       [-1.11846186, -1.04617461, -0.9335314 , ..., -0.94187531,
        -0.60054493, -0.89814704],
       ...,
       [ 0.2167297 ,  0.05974038,  0.41845307, ...,  0.5200759 ,
        -0.19916313,  0.20267869],
       [ 2.65677087,  2.85071455,  1.72459179, ...,  0.46016599,
        -0.10573942,  0.31938169],
       [ 0.21043028,  0.15595506,  0.32574587, ...,  0.39384568,
        -0.10392147,  0.2957848 ]])

In [27]:
import pickle

# Save the scaler objects
with open('StandardScaler_all.pkl', 'wb') as file:
    pickle.dump(scaler_all, file)

with open('StandardScaler_audio.pkl', 'wb') as file:
    pickle.dump(scaler_audio, file)
    
with open('StandardScaler_vision.pkl', 'wb') as file:
    pickle.dump(scaler_vision, file)

## ML training, with hyperparameter optimization, cross-validation

Predicts the melt pool temperature based on features of the other sensing models, iterating over the algorithms:

- K nearest neighbor classification: 'KNN'

- Decision Tree classification: 'DT'

- Adaptive Boosting classification: 'AdaBoost'

- Random Forests classification 'RF'
  
- Gaussian Process classification 'GPR'
  
- Support Vector classification 'SVR'
  
- Gradient Boosting classification 'GB'
  
- Nerual Network classification 'NN'

In [28]:
# class_weight_computed = {
#     class_names[0]: class_weights_all[0], 
#     class_names[1]: class_weights_all[1], 
#     class_names[2]: class_weights_all[2], 
#     class_names[3]: class_weights_all[3],
# }
# print (class_weight_computed)

In [29]:
class_weight_computed = {
    0: class_weights[0], 
    1: class_weights[1], 
    2: class_weights[2], 
    3: class_weights[3],
}
print (class_weight_computed)

{0: 1.2023104752507812, 1: 1.8310167793638867, 2: 1.4722613773660893, 3: 0.5146955297430482}


## Step1: Hyperparameter tuning: RandomizedSearchCV

- Considering the feature space and the data volume, RandomizedSearchCV is much more faster to do.
- Get a initial result for hyperparameter values

In [30]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
import numpy as np

n_iter_search = 20

# -------------------------1 For KNN:-----------------------
knn_distributions = {
    'n_neighbors': sp_randint(1, 50),
    'weights': ['uniform', 'distance']
}
knn_search = RandomizedSearchCV(neighbors.KNeighborsClassifier(), knn_distributions, n_iter=n_iter_search, 
                                scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)

# ------------------2 For Decision Trees:-----------------------
dt_distributions = {
    'min_samples_split': sp_randint(2, 20),
    'min_samples_leaf': sp_randint(1, 20),
    'max_depth': sp_randint(1, 50),
    'ccp_alpha': uniform(0.0, 0.05)
}
dt_search = RandomizedSearchCV(tree.DecisionTreeClassifier(class_weight=class_weight_computed), dt_distributions,
                               n_iter=n_iter_search, scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)

# ------------- 3 for Logistic Regression:----------
lr_distributions = {
    'solver': ['lbfgs', 'sag'],
    'penalty': ['l2'],
    'C': uniform(0.01, 10),
    'max_iter': sp_randint(1000, 10000)
}
lr_search = RandomizedSearchCV(LogisticRegression(class_weight=class_weight_computed), lr_distributions,
                               n_iter=n_iter_search, scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)


# ---------------4 For Random Forest:-------------------
rf_distributions = {
    'n_estimators': sp_randint(1, 500),
    'min_samples_split': sp_randint(2, 20),
    'min_samples_leaf': sp_randint(1, 20),
    'max_depth': sp_randint(1, 100),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}
rf_search = RandomizedSearchCV(RandomForestClassifier(class_weight=class_weight_computed), rf_distributions,
                               n_iter=n_iter_search, scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)


# ------------- 5 for BalancedRandomForestClassifier -----
brf_distributions = {
    'n_estimators': sp_randint(1, 500),
    'min_samples_split': sp_randint(2, 20),
    'min_samples_leaf': sp_randint(1, 20),
    'max_depth': sp_randint(1, 50),
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}
brf_search = RandomizedSearchCV(BalancedRandomForestClassifier(), brf_distributions, 
                                n_iter=n_iter_search, scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)


# ------------- 6 for RUSBoostClassifier---------------
rusboost_distributions = {
    'n_estimators': sp_randint(1, 500),
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 1.0]
}
rusboost_search = RandomizedSearchCV(RUSBoostClassifier(), rusboost_distributions,
                                     n_iter=n_iter_search, scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)


# ------------------7 For AdaBoost:----------------------
ada_distributions = {
    'n_estimators': sp_randint(1, 500),
    'algorithm': ['SAMME', 'SAMME.R'],
    'learning_rate': uniform(0.5, 1.5)
}
ada_search = RandomizedSearchCV(AdaBoostClassifier(), ada_distributions, n_iter=n_iter_search,
                                scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)


# # ------------8 For Gaussian Process:------------
# gp_distributions = {
#     'kernel': [1.0 * RBF(1.0)]
# }
# gp_search = RandomizedSearchCV(GaussianProcessClassifier(), gp_distributions, n_iter=n_iter_search,
#                                scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)


# ------------ 9 For Neural Network:-----------
nn_distributions = {
    'hidden_layer_sizes': [(sp_randint(16, 256).rvs(), sp_randint(16, 256).rvs(), sp_randint(16, 256).rvs()) for _ in range(10)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': uniform(0.001, 0.1),
    'learning_rate': ['adaptive'],
    'max_iter': sp_randint(1000, 20000),
    'early_stopping': [True],
    'validation_fraction': [0.1]
}
nn_search = RandomizedSearchCV(MLPClassifier(), nn_distributions, n_iter=n_iter_search, 
                               scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)


## ------------ 10 For Xboost:-----------
xgb_distributions = {
    'n_estimators': sp_randint(50, 500),
    'learning_rate': uniform(0.01, 0.5),
    'subsample': uniform(0.5, 0.5),
    'max_depth': list(range(1, 50)),
    'colsample_bytree': uniform(0.5, 0.5),
    'min_child_weight': list(range(1, 20)),
    'gamma': uniform(0, 2)
}
xgb_search = RandomizedSearchCV(xgb.XGBClassifier(objective='multi:softmax'), xgb_distributions, 
                                n_iter=n_iter_search, scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42
)


# ------------ 11 for LightGBM -----------
lgb_distributions = {
    'n_estimators': sp_randint(50, 500),
    'learning_rate': uniform(0.01, 0.5),
    'subsample': uniform(0.5, 0.5),
    'num_leaves': sp_randint(20, 150),
    'max_depth': sp_randint(1, 50),
    'colsample_bytree': uniform(0.5, 0.5),
    'min_child_weight': sp_randint(1, 20),
    'reg_alpha': uniform(0, 2),
    'reg_lambda': uniform(0, 2),
}

lgb_search = RandomizedSearchCV(lgb.LGBMClassifier(objective='multiclass', metric='multi_logloss'), 
                                lgb_distributions, n_iter=n_iter_search, scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)


# ----------------12 For Support Vector Machine:-----------------
svm_distributions = {
    'C': uniform(0.1, 1000),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'degree': sp_randint(1, 10),
    'gamma': uniform(1e-4, 1e-1)
}
svm_search = RandomizedSearchCV(SVC(class_weight=class_weight_computed), svm_distributions, 
                                n_iter=n_iter_search, scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)


# ---------  13 For GradientBoostingClassifier:---------------------
gboost_distributions = {
    'n_estimators': sp_randint(1, 500),
    'learning_rate': uniform(0.01, 0.5),
    'subsample': uniform(0.5, 0.5),
    'max_depth': sp_randint(1, 50),
    'min_samples_split': sp_randint(2, 20),
    'min_samples_leaf': sp_randint(1, 20),
    'n_iter_no_change': [10],
    'tol': [1e-4]
}
gboost_search = RandomizedSearchCV(GradientBoostingClassifier(), gboost_distributions, 
                                   n_iter=n_iter_search, scoring="accuracy", cv=5, n_jobs=-1, verbose=3, random_state=42)

In [31]:
len(X_all_train_transformed)*0.5

14622.5

### RandomizedSearchCV Initialization & Model Fitting (ALL features)

In [32]:
# Sample a subset of your data for faster training
n_samples = 14600 # 50% of the total data
X_subset, y_subset = resample(X_all_train_transformed, Y_train_encoded, n_samples=n_samples, random_state=42)

# Sample a subset of your data for faster training
n_samples = 3000  # 10% of the total data
X_sample, y_sample = resample(X_all_train_transformed, Y_train_encoded, n_samples=n_samples, random_state=42)

In [ ]:
from scipy.stats import randint, uniform


# 1. KNN
knn_search.fit(X_subset, y_subset)

# 2. Decision Tree
dt_search.fit(X_subset, y_subset)

# 3. Logistic Regression:
lr_search.fit(X_subset, y_subset)

# 4. Random Forest
rf_search.fit(X_subset, y_subset)

# 5. Balanced Forest
brf_search.fit(X_subset, y_subset)

# 6. RUSBoost
rusboost_search.fit(X_subset, y_subset)

# 7. AdaBoost 
ada_search.fit(X_subset, y_subset)

# 8. Gaussian Process 
gp_search.fit(X_sample, y_sample)

# 9. Neural Network:
nn_search.fit(X_subset, y_subset)

# 10. XGBoost:
xgb_search.fit(X_sample, y_sample)

# 11. LightGBM
lgb_search.fit(X_sample, y_sample)

# 12. SVM:
svm_search.fit(X_subset, y_subset)

# 13. GradientBoostingClassifier
gboost_search.fit(X_sample, y_sample)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits
Fitting 5 folds for each of 20 candidates, totalling 100 fits


/home/chenlequn/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/chenlequn/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/chenlequn/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/chenlequn/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/chenlequn/anaconda3/envs/torch/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


### Best hyperparameters of each models

In [ ]:
# 1
print("Best parameters for KNN: ", knn_search.best_params_)
print("Best cross-validation score for KNN: {:.2f}\n".format(knn_search.best_score_))

# 2
print("Best parameters for DT: ", dt_search.best_params_)
print("Best cross-validation score for DT: {:.2f}\n".format(dt_search.best_score_))

# 3
print("Best parameters for LR: ", lr_search.best_params_)
print("Best cross-validation score for LR: {:.2f}\n".format(lr_search.best_score_))

# 4
print("Best parameters for RF: ", rf_search.best_params_)
print("Best cross-validation score for RF: {:.2f}\n".format(rf_search.best_score_))

# 5
print("Best parameters for BalancedRandomForestClassifier: ", brf_search.best_params_)
print("Best cross-validation score for BalancedRandomForestClassifier: {:.2f}\n".format(brf_search.best_score_))

# 6
print("Best parameters for RUSBoostClassifier: ", rusboost_search.best_params_)
print("Best cross-validation score for RUSBoostClassifier: {:.2f}\n".format(rusboost_search.best_score_))

# 7 
print("Best parameters for Ada: ", ada_search.best_params_)
print("Best cross-validation score for Ada: {:.2f}\n".format(ada_search.best_score_))

# 8
print("Best parameters for GP: ", gp_search.best_params_)
print("Best cross-validation score for GP: {:.2f}\n".format(gp_search.best_score_))

# 9
print("Best parameters for NN: ", nn_search.best_params_)
print("Best cross-validation score for NN: {:.2f}\n".format(nn_search.best_score_))

# 10
print("Best parameters for XGBoost: ", xgb_search.best_params_)
print("Best cross-validation score for XGBoost: {:.2f}\n".format(xgb_search.best_score_))

# 11
print("Best parameters for LightGBM: ", lgb_search.best_params_)
print("Best cross-validation score for LightGBM: {:.2f}\n".format(lgb_search.best_score_))

# 12
print("Best parameters for SVM: ", svm_search.best_params_)
print("Best cross-validation score for SVM: {:.2f}\n".format(svm_search.best_score_))

# 13. GradientBoostingClassifier
print("Best parameters for GB: ", gboost_search.best_params_)
print("Best cross-validation score for GB: {:.2f}\n".format(gboost_search.best_score_))

## Step 2: Train the model using the best hyperparameter on the trianing dataset

### Best Hyperparameters (for class_v2, three categories)

In [ ]:
len(X_all_train_transformed)*1

In [ ]:
n_samples = 29245 # 100% of the total data
X_subset, y_subset = resample(X_all_train_transformed, Y_train_encoded, n_samples=n_samples, random_state=42)

In [ ]:
# 1. AdaBoost
ada_model = AdaBoostClassifier(algorithm='SAMME', learning_rate=0.5696756190799965, n_estimators=307)
ada_model.fit(X_subset, y_subset)
ada_val_score = ada_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for AdaBoost: {ada_val_score:.2f}")


# 2. Decision Tree
dt_model = tree.DecisionTreeClassifier(ccp_alpha=3.893829205071642e-05, max_depth=21,
                                       min_samples_leaf=1, min_samples_split=13)
dt_model.fit(X_subset, y_subset)
dt_val_score = dt_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for DT: {dt_val_score:.2f}")

# 3. Logistic Regression
lr_model = LogisticRegression(C=5.152344384136116, max_iter=2528, penalty='l2', solver='lbfgs')
lr_model.fit(X_subset, y_subset)
lr_val_score = lr_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for LR: {lr_val_score:.2f}")


# 4. Random Forest
rf_model = RandomForestClassifier(bootstrap=False, max_depth=64, max_features='sqrt',
                                  min_samples_leaf=3, min_samples_split=6, n_estimators=307)
rf_model.fit(X_subset, y_subset)
rf_val_score = rf_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for RandomForest: {rf_val_score:.2f}")


# 5. Balanced Random Forest
brf_model = BalancedRandomForestClassifier(bootstrap=False, max_depth=23, max_features='sqrt',
                                           min_samples_leaf=8, min_samples_split=17, n_estimators=237)
brf_model.fit(X_subset, y_subset)
brf_val_score = brf_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for Balanced Random Forest: {brf_val_score:.2f}")


# 6. RUSBoostClassifier
rus_model = RUSBoostClassifier(learning_rate=0.1, n_estimators=150)
rus_model.fit(X_subset, y_subset)
rus_val_score = rus_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for RUSBoostClassifier: {rus_val_score:.2f}")


# 7. KNN
knn_model = neighbors.KNeighborsClassifier(n_neighbors=12, weights='distance')
knn_model.fit(X_subset, y_subset)
knn_val_score = knn_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for KNN: {knn_val_score:.2f}")


# 8. Gaussian Process
# kernel = 1**2 * RBF(length_scale=1)
# gp_model = GaussianProcessClassifier(kernel=kernel)
# gp_model.fit(X_subset, y_subset)
# gp_val_score = gp_model.score(X_all_val_transformed, Y_val_encoded)
# print(f"Validation score for GP: {gp_val_score:.2f}")



# 9. Neural Network
nn_model = MLPClassifier(activation='relu', alpha=0.07653614103176526, early_stopping=True,
                         hidden_layer_sizes=(229, 255, 168), learning_rate='adaptive',
                         max_iter=11805, solver='adam', validation_fraction=0.1)
nn_model.fit(X_subset, y_subset)
nn_val_score = nn_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for NN: {nn_val_score:.2f}")


# 10. XGBoost
xgb_model = xgb.XGBClassifier(colsample_bytree=0.6154469128110744, gamma=0.48205093205202343,
                              learning_rate=0.3516317594127291, max_depth=44,
                              min_child_weight=8, n_estimators=480, subsample=0.5866823267538861) #objective='multiclass', metric='multi_logloss',
xgb_model.fit(X_subset, y_subset)
xgb_val_score = xgb_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for XGBoost: {xgb_val_score:.2f}")


# 11. LightGBM
lgb_model = lgb.LGBMClassifier(colsample_bytree=0.7956488938538635, learning_rate=0.1473608964950321,
                               max_depth=35, min_child_weight=1, n_estimators=276, num_leaves=120,
                               reg_alpha=1.9434241907782075, reg_lambda=1.6978276485321677, subsample=0.8608647605824367) #objective='multiclass', metric='multi_logloss',
lgb_model.fit(X_subset, y_subset)
lgb_val_score = lgb_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for LightGBM: {lgb_val_score:.2f}")


# 12. SVM
svm_model = SVC(C=524.8564316322379, degree=9, gamma=0.029222914019804192, kernel='rbf')
svm_model.fit(X_subset, y_subset)
svm_val_score = svm_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for SVM: {svm_val_score:.2f}")


# 13. Gradient Boosting
gb_model = GradientBoostingClassifier(learning_rate=0.2259725093210579, max_depth=49,
                                      min_samples_leaf=10, min_samples_split=17,
                                      n_estimators=271, n_iter_no_change=10, subsample=0.728034992108518, tol=0.0001)
gb_model.fit(X_subset, y_subset)
gb_val_score = gb_model.score(X_all_val_transformed, Y_val_encoded)
print(f"Validation score for GB: {gb_val_score:.2f}")

## Performance analysis on the test set

In [ ]:
accuracy_mean_list = []
accuracy_std_list = []

auc_mean_list = []
auc_std_list = []

In [ ]:
labels_categorical_

In [ ]:
# Iterate over implemented models
for modeltoggle in ["KNN", "DT", "LR", "RF", "BalancedRandomForest", "RUSBoostClassifier", "AdaBoost", "NN", "XGBoost", "SVM", "GB", "LightGBM"]:


    title = "Defect Prediction "
    labels_categorical_=(le.classes_).tolist() #['Defect-free', 'Keyhole pores', 'Laser defocus', 'Laser-off']
    n_classes = len(labels_categorical_)
    X = X_all_test_transformed
    Y = Y_test_encoded

    # One-hot-encode the ouput
    Y_test_ohe = label_binarize(Y_test, classes= labels_categorical_)

    if modeltoggle == 'KNN':
        model = knn_model
    elif modeltoggle == 'DT':
        model = dt_model
    elif modeltoggle == 'LR':
        model = lr_model
    elif modeltoggle == 'RF':
        model = rf_model
    elif modeltoggle == 'BalancedRandomForest':
        model = brf_model
    elif modeltoggle == 'RUSBoostClassifier':
        model = rus_model
    elif modeltoggle == 'AdaBoost':
        model = ada_model
    elif modeltoggle == 'GP':
        model = gp_model
    elif modeltoggle == 'NN':
        model = nn_model
    elif modeltoggle == 'XGBoost':
        model = xgb_model
    elif modeltoggle == 'SVM':
        model = svm_model
    elif modeltoggle == 'GB':
        model = gb_model
    elif modeltoggle == 'LightGBM':
        model = lgb_model

    ################--------------------------------- confusion matrix ------------------#####################
    ################---------------------------------------------------------------------#####################
    # for calculating the confusion matrix
    Y_predict = model.predict(X)
    # Category/ label that was wrongly classified by the model: View the class with WRONG classificaiton 
    pred_result = list(zip(Y, Y_predict))
    result_df = pd.DataFrame(pred_result, columns = ['label', 'pred_label'])
    result_df = result_df[result_df['label'] != result_df['pred_label']]
    print(classification_report(Y, Y_predict))
    
    conf_matrix(Y, Y_predict, classes=labels_categorical_)
    save_fig("confusion_"+ modeltoggle + "_" + "ALL")
    
    
    ################------------ Testing performance on k-fold on the model -------------#####################
    ################---------------------------------------------------------------------#####################
    kf = KFold(n_splits=5)
    skf = StratifiedKFold(n_splits=5)
    trains = []
    tests = []
    
    # for train, test in kf.split(X):
    for train, test in skf.split(X, Y):

        X_shuffle, y_shuffle = shuffle(X, Y, random_state=0)
        y_shuffle = np.array(y_shuffle)

        # X_cv_train_before = X_shuffle[train]
        # X_cv_test_before = X_shuffle[test]
        X_train_cv = X_shuffle[train]
        X_test_cv = X_shuffle[test]
        y_train_cv = y_shuffle[train]
        y_test_cv = y_shuffle[test]
        # clf.fit(X_train_cv, y_train_cv)
        # trains.append(clf.score(X_train_cv, y_train_cv))
        tests.append(model.score(X_test_cv, y_test_cv))
    

    ################--------------- AUC-ROC score measurement --------------------#####################

    # ----------------Learn to predict each class against the other------------------------
    # classifier = OneVsRestClassifier(model)
    if modeltoggle == 'SVM':
        # y_score = classifier.fit(X_train, y_train).decision_function(X_test)
        y_score = model.decision_function(X)
    else:
        # y_score = classifier.fit(X_train, Y_train_ohe).predict_proba(X_test)
        y_score = model.predict_proba(X)
    
    # ---------------Compute ROC curve and ROC area for each class-------------------------
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(Y_test_ohe[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(Y_test_ohe.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at this points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes

    fpr["macro"] = all_fpr
    tpr["macro"] = mean_tpr
    roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

    # ----------------------------------Plot all ROC curves-------------------------------
    plt.figure(figsize = (4,3), dpi = 300)
    widths = 2
    ax = gca()
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(widths)

        tick_width = 1.5
    plt.tick_params(direction = 'in', width = tick_width)
    
         
    #---------------------------(1) micro and macro ROC curve---------------------------
    plt.plot(fpr["micro"], tpr["micro"],
             label=modeltoggle + ' - micro-average ROC curve (AUC = {0:0.2f})'
                   ''.format(roc_auc["micro"]),
             color='red', linestyle=':', linewidth=2, alpha = 0.8) #deeppink, midnightblue

    plt.plot(fpr["macro"], tpr["macro"],
             label=modeltoggle + ' - macro-average ROC curve (AUC = {0:0.2f})'
                   ''.format(roc_auc["macro"]),
             color='navy', linestyle=':', linewidth=2, alpha = 0.8) #navy, gold
    
    #---------------------------(2) ROC curve for each class---------------------------
    colors = cycle(["aqua", "darkblue", "darkorange", "red", 'green', 'silver', 'yellow', 'olive'])
    # colors = cycle(['0.45', 'steelblue',  'olive', 'silver'])
    for i, color in zip(range(n_classes), colors):
        plt.plot(fpr[i],tpr[i],color=color,
                 lw=1, alpha = 0.8,
                 label=modeltoggle + " ROC curve of class \"{0}\" (area = {1:0.2f})".format(labels_categorical_[i], roc_auc[i])) 
        
    plt.plot([0, 1], [0, 1], "k--", lw=2)
    plt.xlim([-0.02, 1.02])
    plt.ylim([-0.02, 1.02])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    # plt.title("ROC curve")
    plt.legend(loc="lower right",  fontsize = '6', frameon = True, framealpha=0.8)
    # plt.show()
    plt.grid(linestyle='--', alpha=0.5, linewidth=0.8)
    
    #---------------------------------Aggregate information----------------------------------
    tests = np.array(tests)

    test_accuracy_mean = tests.mean()
    test_accuracy_std = tests.std()
    
    auc_score_list = []
    auc_score_list.append(auc(fpr["micro"], tpr["micro"]))
    
    auc_score_array = np.array(auc_score_list)
    auc_mean = auc_score_array.mean()
    auc_std = auc_score_array.std()
    
    
    # store into list
    auc_mean_list.append(auc_mean)
    auc_std_list.append(auc_std)
    accuracy_mean_list.append(test_accuracy_mean)
    accuracy_std_list.append(test_accuracy_std)
    
    print('Test Accuracy (cross-validation) for' , modeltoggle, '= {:.5f} ± {:.5f}'.format(test_accuracy_mean, test_accuracy_std))
    print('micro-averaging AUC for' , modeltoggle, '= {:.5f} ± {:.5f}'.format(auc_mean, auc_std))
    save_fig("ROC_" + modeltoggle + "_" + "audio_visual")

## Save trained model into folder

In [ ]:
from joblib import dump, load
import joblib

KNN_file = os.path.join(model_path, 'KNN.joblib')
DT_file = os.path.join(model_path, 'DT.joblib')
AdaBoost_file = os.path.join(model_path, 'AdaBoost.joblib')
RF_file = os.path.join(model_path, 'RF.joblib')
BalancedRandomForest_file = os.path.join(model_path, 'BalancedRandomForest.joblib')
LightGBM_file = os.path.join(model_path, 'LightGBM.joblib')
XGBoost_file = os.path.join(model_path, 'XGBoost.joblib')
LR_file = os.path.join(model_path, 'LR.joblib')
SVM_file = os.path.join(model_path, 'SVM.joblib')
GB_file = os.path.join(model_path, 'GB.joblib')
RUSBoostClassifier_file = os.path.join(model_path, 'RUSBoostClassifier.joblib')
NN_file = os.path.join(model_path, 'NN.joblib')

# Define the paths
paths = {
    'KNN': KNN_file,
    'DT': DT_file,
    'AdaBoost': AdaBoost_file,
    'RF': RF_file,
    'BalancedRandomForest': BalancedRandomForest_file,
    'LightGBM': LightGBM_file,
    'XGBoost': XGBoost_file,
    'LR': LR_file,
    'SVM': SVM_file,
    'GB': GB_file,
    'RUSBoostClassifier': RUSBoostClassifier_file,
    # 'GP': GP_file, 
    'NN': NN_file
}

# Define the models
models = {
    'KNN': knn_model,
    'DT': dt_model,
    'AdaBoost': ada_model,
    'RF': rf_model,
    'BalancedRandomForest': brf_model,
    'LightGBM': lgb_model,
    'XGBoost': xgb_model,
    'LR': lr_model,
    'SVM': svm_model,  
    'GB': gb_model,  
    'RUSBoostClassifier': rus_model,
    # 'GP': gp_model,  
    'NN': nn_model  
}

# Save models
for name, model in models.items():
    dump(model, paths[name])

In [ ]:
# load the model from disk
loaded_model = joblib.load(open(RF_file, 'rb'))
result = loaded_model.score(X_all_test_transformed, Y_test_encoded)
print(result)

In [ ]:
def plot_model_sizes(paths, fontsize=14):
    model_sizes = {}
    for name, path in paths.items():
        model_sizes[name] = os.path.getsize(path) // 1024  # Size in KB

    sorted_model_sizes = {k: v for k, v in sorted(model_sizes.items(), key=lambda item: item[1])}

    plt.figure(figsize=(10, 6))
    plt.barh(list(sorted_model_sizes.keys()), list(sorted_model_sizes.values()), 
             color='skyblue', edgecolor='black', linewidth=0.5)
    plt.xlabel('Model Size (KB)', fontsize=fontsize+6, labelpad=10)
    plt.ylabel('Model', fontsize=fontsize+6)
    plt.xticks(fontsize=fontsize, rotation=45, ha='right')
    plt.yticks(fontsize=fontsize)
    plt.grid(which='both', linestyle='--', linewidth=0.5)
    # plt.show()

# Example usage:
plot_model_sizes(paths)
save_fig("model size comparisons")

## Model performance statistics

BalancedRandomForest_fileTo evaluate the effectiveness of the classification model, metrics used to assess are:

1. **$Precision = TP/(FP+TP)$**

**Precision** provides the *positive predictive value, the proportion of samples that belong in category $x$ that are correctly placed in category $x$*. Example, among all predicted defect, how many did I predict correctly? High precision is with low FP (predict to be cracks but actually is not).

2. **$f1 = 2*(Precision*Recall)/(Precision + Recall)$**

**Weighted average for precision and recall**. This score takes both FN and FP into account as $Recall = TP/(FN+TP)$. This is better metric for evaluating an uneven class distribution problem, which is the case in this project. 

3. $Accuracy = (TP+TN)/(All predict)$

Model achieve pretty high **>80%** overall accuracy.

The model able to classify well for defects sounds. However, it did badly on the detecting ***cracks***

- In conclusion, the initial model without Hyper-parameter tuning did pretty well in general. But it does not outperform the traditional ML algorithms. And it did poorly in classifying the cracking regimes.

In [ ]:
models_list = ["KNN", "DT", "LR", "RF", "BalancedRandomForest", "RUSBoostClassifier", "AdaBoost", "NN", "XGBoost", "SVM", "GB", "LightGBM"]

In [ ]:
accuracy_mean_list

In [ ]:
def plot_model_metrics(models, mean_list, std_list, metric_name, color, 
                       bar_width=0.6, edge_width=1, fontsize=15, 
                       labelpad=10, figsize=(10, 8), dpi=300, add_baseline=False):
    
    # If baseline model is to be added (only for accuracy)
    if add_baseline:
        models.append("baseline model")
        mean_list.append(0.26)
        std_list.append(0)
    
    # Sorting by mean values
    sorted_indices = np.argsort(mean_list)
    sorted_models = np.array(models)[sorted_indices]
    sorted_means = np.array(mean_list)[sorted_indices]
    sorted_stds = np.array(std_list)[sorted_indices]
    
    # Plotting
    plt.figure(figsize=figsize, dpi=dpi)
    plt.bar(sorted_models, sorted_means, yerr=sorted_stds, 
            capsize=10, color=color, width=bar_width, 
            edgecolor='black', linewidth=edge_width)
    
    plt.ylabel(metric_name, fontsize=fontsize, labelpad=labelpad)
    plt.title(f'Model {metric_name} Comparison', fontsize=fontsize)
    
    plt.xticks(fontsize=fontsize, rotation=45,  ha='right')
    plt.yticks(fontsize=fontsize)
    plt.xlabel("Models", fontsize=fontsize, labelpad=labelpad)
    
    # Add dotted grid
    plt.grid(which='both', linestyle='--', linewidth=0.5)
    
    # Show ticks outwards
    plt.tick_params(direction='out', length=6, width=2, colors='black')
    
    plt.tight_layout()
    plt.ylim([0, 1.02])
    plt.yticks([0.0, 0.2, 0.4, 0.6, 0.8, 0.9, 1.0])
    # plt.show()

# Plotting Accuracy with improvements
plot_model_metrics(models_list.copy(), accuracy_mean_list.copy(), accuracy_std_list.copy(), 
                   "Accuracy", 'skyblue', add_baseline=False)
save_fig ("Accuracy-all(audio_visual)")
# Plotting AUC with improvements
plot_model_metrics(models_list.copy(), auc_mean_list.copy(), auc_std_list.copy(), 
                   "AUC Score", 'lightcoral')
save_fig ("AUC-all(audio_visual)")

## Analyze the inference time

1. Warm-Up Runs: A few 'warm-up' runs are often performed before the timing measurements to ensure that any lazy loading, caching, or other one-time operations have been completed.

2. High-Resolution Timers: High-resolution timers (such as Python's time.perf_counter() or C++'s std::chrono) are used to measure the time elapsed.

3. Excluding Data Loading: The time taken for just the model to make a prediction is measured, excluding any data loading or pre-processing time unless that is also a part of what is being evaluated.

4. GPU/CPU Synchronization: If using a GPU, it's important to make sure to synchronize the CPU and GPU computations to accurately measure the time taken.



In [ ]:
import time

X_sample = X_all_test_transformed[10].reshape(1, -1)

# Dictionary to store inference times
inference_times = {}

# Number of runs
n_runs = 50

# Loop through all models and measure inference time
for model_name, model in models.items():
    times = []
    for _ in range(n_runs):
        start_time = time.perf_counter()
        predictions = model.predict(X_sample)
        end_time = time.perf_counter()
        times.append(end_time - start_time)
        
    # Calculate average and standard deviation
    avg_time = sum(times) / n_runs
    std_time = (sum((x - avg_time) ** 2 for x in times) / n_runs) ** 0.5
    
    inference_times[model_name] = (avg_time, std_time)
    print(f"Inference time for {model_name}: {avg_time:.6f} ± {std_time:.6f} seconds")

# Optional: Sort by average inference time
sorted_times = {k: v for k, v in sorted(inference_times.items(), key=lambda item: item[1][0])}
print("\nModels sorted by inference time:", sorted_times)

In [ ]:
def plot_inference_time(models, inference_times, figsize=(10, 8), dpi=300):
    # Unpack times and standard deviations
    model_names = list(inference_times.keys())
    avg_times = [x[0] * 1e3 for x in inference_times.values()]  # Convert to milliseconds
    std_times = [x[1] * 1e3 for x in inference_times.values()]  # Convert to milliseconds
    
    # Sorting by mean values
    sorted_indices = np.argsort(avg_times)
    sorted_models = np.array(model_names)[sorted_indices]
    sorted_means = np.array(avg_times)[sorted_indices]
    sorted_stds = np.array(std_times)[sorted_indices]
    
    # Plotting
    plt.figure(figsize=figsize, dpi=dpi)

    # Before plotting, adjust std deviation if it leads to negative values
    corrected_stds = [min(std, mean) for std, mean in zip(sorted_stds, sorted_means)]
    
    plt.bar(sorted_models, sorted_means, yerr=corrected_stds, 
            capsize=10, color='skyblue', width=0.6, 
            edgecolor='black', linewidth=1)
    
    plt.ylabel('Inference Time (ms)', fontsize=15, labelpad=10)
    plt.title('Model Inference Time Comparison', fontsize=15)
    
    plt.xticks(fontsize=15, rotation=45, ha='right')
    plt.yticks(fontsize=15)
    plt.xlabel("Models", fontsize=15, labelpad=10)
    
    # Add dotted grid
    plt.grid(which='both', linestyle='--', linewidth=0.5)
    
    # Show ticks outwards
    plt.tick_params(direction='out', length=6, width=2, colors='black')
    
    plt.tight_layout()
    
# Plot the inference times
plot_inference_time(models, inference_times)
save_fig("inference_time")

In [ ]:
# Remove the models "AdaBoost" and "GB"
inference_times.pop("RF", None)
inference_times.pop("BalancedRandomForest", None)
inference_times.pop("RUSBoostClassifier", None)
inference_times.pop("AdaBoost", None)
inference_times.pop("KNN", None)
inference_times.pop("XGBoost", None)
# Extracting data for plotting
model_names = list(inference_times.keys())

plot_inference_time(model_names, inference_times)
save_fig("inference_time_selected")